In [55]:
from sklearn.decomposition import PCA 
from sklearn.svm import SVC 
from sklearn.utils import resample
import os
import random
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from PIL import Image
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from PIL import Image
from skimage import color, filters
import cv2
import matplotlib.pyplot as plt
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
import urllib.request
from skimage import measure

In [56]:
target_classes = ['go', 'stop', 'warning']
color_map = {'go':'green', 'stop':'red', 'warning':'yellow'}
rgb_color_map = {'go': (0, 255, 0), 'stop': (255, 0, 0), 'warning': (255, 255, 0)}

train_folder_list = [
    'dayTrain',
#     'daySequence1',
#     'daySequence2',
#     'sample-dayClip6',
    'nightTrain',
#     'nightSequence1',
#     'nightSequence2',
#     'sample-nightClip1',
]

n_samples_per_class = 1000

In [57]:
def get_annotarion_dataframe(train_data_folders):
    data_base_path = '../input/lisa-traffic-light-dataset/'
    annotation_list = list()
    for folder in [folder + '/' for folder in train_data_folders if os.listdir(data_base_path)]:
        annotation_path = ''
        if 'sample' not in folder:
            annotation_path = data_base_path + 'Annotations/Annotations/' + folder
        else:
            annotation_path = data_base_path + folder*2
        image_frame_path = data_base_path + folder*2
        
        df = pd.DataFrame()
        if 'Clip' in os.listdir(annotation_path)[0]:
            clip_list = os.listdir(annotation_path)
            for clip_folder in clip_list:
                df = pd.read_csv(annotation_path + clip_folder +  '/frameAnnotationsBOX.csv', sep=";")
                df['image_path'] = image_frame_path + clip_folder + '/frames/'
                annotation_list.append(df)
        else:
            df = pd.read_csv(annotation_path +  'frameAnnotationsBOX.csv', sep=";")
            df['image_path'] = image_frame_path + 'frames/'
            annotation_list.append(df)
        
    df = pd.concat(annotation_list)
    df = df.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
    df.columns = ['filename', 'target', 'x1', 'y1', 'x2', 'y2', 'image_path']
    df = df[df['target'].isin(target_classes)]
    df['filename'] = df['filename'].apply(lambda filename: re.findall("\/([\d\w-]*.jpg)", filename)[0])
    df = df.drop_duplicates().reset_index(drop=True)
    return df

In [58]:
train_annotation_df = get_annotarion_dataframe(train_folder_list)

target_classes = train_annotation_df['target'].unique()
target_classes.sort()

train_annotation_df

,filename,target,x1,y1,x2,y2,image_path
0,dayClip1--00000.jpg,go,698,333,710,358,../input/lisa-traffic-light-dataset/dayTrain/d...
1,dayClip1--00000.jpg,go,846,391,858,411,../input/lisa-traffic-light-dataset/dayTrain/d...
2,dayClip1--00001.jpg,go,698,337,710,357,../input/lisa-traffic-light-dataset/dayTrain/d...
3,dayClip1--00001.jpg,go,847,390,859,410,../input/lisa-traffic-light-dataset/dayTrain/d...
4,dayClip1--00002.jpg,go,698,331,710,356,../input/lisa-traffic-light-dataset/dayTrain/d...
...,...,...,...,...,...,...,...
42565,nightClip5--01450.jpg,go,937,221,976,290,../input/lisa-traffic-light-dataset/nightTrain...
42566,nightClip5--01451.jpg,go,467,36,512,117,../input/lisa-traffic-light-dataset/nightTrain...
42567,nightClip5--01451.jpg,go,937,219,976,288,../input/lisa-traffic-light-dataset/nightTrain...
42568,nightClip5--01452.jpg,go,460,33,506,114,../input/lisa-traffic-light-dataset/nightTrain...


In [59]:
def crop_image(image, x1, y1, x2, y2):
    cropped_image = image[y1:y2, x1:x2]
    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    
    return cropped_image

In [60]:
def generate_bouding_boxes(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convert the image to a NumPy array
    image_array = np.array(image)

    # Convert the image array to L*a*b* color space
    lab_image = color.rgb2lab(image_array)

    # Extract the L, a, and b channels
    L = lab_image[:, :, 0]
    a = lab_image[:, :, 1]
    b = lab_image[:, :, 2]


    # Calculate the RGYB channel
    RGYB = 0.7*L * (1.0*a + 0.7*b)
    YELLOW = 0.7*L * (0.7*a + b)


    # Create a mask for red and green blobs
    red_mask = np.logical_and(RGYB > 2000, L <= 80)
    green_mask = np.logical_and(RGYB < -2000, L <= 80)
    yellow_mask = np.logical_and(YELLOW > 1000, RGYB <= 2000)

    # Apply the masks to the original image
    red_blobs = np.zeros_like(image_array)
    red_blobs[red_mask] = image_array[red_mask]
    green_blobs = np.zeros_like(image_array)
    green_blobs[green_mask] = image_array[green_mask]
    yellow_blobs = np.zeros_like(image_array)
    yellow_blobs[yellow_mask] = image_array[yellow_mask]

    # Convert the red and green blobs to grayscale
    red_gray = color.rgb2gray(red_blobs)
    green_gray = color.rgb2gray(green_blobs)
    yellow_gray = color.rgb2gray(yellow_blobs)

    red_gray = np.where(red_gray > 0, 255, 0).astype(np.uint8)
    green_gray = np.where(green_gray > 0, 255, 0).astype(np.uint8)
    yellow_gray = np.where(yellow_gray > 0, 255, 0).astype(np.uint8)


    # Use built-in sobel filter in hough circle transform.
    red_edges = red_gray
    green_edges = green_gray
    yellow_edges = yellow_gray

    # Perform Hough circle detection on red edges
    red_radii = np.arange(5, 30)  # Define the expected radius range for red circles
    red_hough = hough_circle(red_edges, red_radii)
    red_accums, red_centers_x, red_centers_y, red_radii = hough_circle_peaks(red_hough, red_radii, threshold=0.90*np.max(red_hough), min_xdistance = 50, min_ydistance = 50)

    # Perform Hough circle detection on green edges
    green_radii = np.arange(5, 30)  # Define the expected radius range for red circles
    green_hough = hough_circle(green_edges, green_radii)
    green_accums, green_centers_x, green_centers_y, green_radii = hough_circle_peaks(green_hough, green_radii, threshold=0.90*np.max(green_hough), min_xdistance=50, min_ydistance = 50)

    # Perform Hough circle detection on yellow edges
    yellow_radii = np.arange(5, 30)  # Define the expected radius range for yellow circles
    yellow_hough = hough_circle(yellow_edges, yellow_radii)
    yellow_accums, yellow_centers_x, yellow_centers_y, yellow_radii = hough_circle_peaks(yellow_hough, yellow_radii, threshold=0.90*np.max(yellow_hough), min_xdistance=50, min_ydistance = 50)

    # Draw detected circles on the original image

    cropped_images = []
    bounding_boxes = []
    labels = []

    red_circles_image = np.copy(image_array)
    for center_y, center_x, radius in zip(red_centers_y, red_centers_x, red_radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        valid_coords = np.logical_and(circy >= 0, circy < red_circles_image.shape[0]) & np.logical_and(circx >= 0, circx < red_circles_image.shape[1])
        red_circles_image[circy[valid_coords], circx[valid_coords]] = (0, 255, 0)  # Red color for circles
        max_x = red_circles_image.shape[1]
        max_y = red_circles_image.shape[0]
        x1, y1, x2, y2 = max(0,center_x - 1.9*radius), max(0, center_y - 1.9*radius), min(max_x, center_x + 1.9*radius), min(max_y, center_y + (7.0)*radius)
        x1, x2, y1, y2 = round(x1), round(x2), round(y1), round(y2) 
        cropped_images.append(crop_image(image_array, x1, y1, x2, y2))
        bounding_boxes.append([x1, y1, x2, y2])
        labels.append("red")


    green_circles_image = np.copy(image_array)
    for center_y, center_x, radius in zip(green_centers_y, green_centers_x, green_radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        valid_coords = np.logical_and(circy >= 0, circy < green_circles_image.shape[0]) & np.logical_and(circx >= 0, circx < green_circles_image.shape[1])
        green_circles_image[circy[valid_coords], circx[valid_coords]] = (255, 0, 0)  # Green color for circles
        max_x = green_circles_image.shape[1]
        max_y = green_circles_image.shape[0]
        x1, y1, x2, y2 = max(0,center_x - 1.9*radius), max(0, center_y - 7.0*radius), min(max_x, center_x + 1.9*radius), min(max_y, center_y + (1.9)*radius)
        x1, x2, y1, y2 = round(x1), round(x2), round(y1), round(y2) 
        cropped_images.append(crop_image(image_array, x1, y1, x2, y2))
        bounding_boxes.append([x1, y1, x2, y2])
        labels.append("green")
        
    yellow_circles_image = np.copy(image_array)
    for center_y, center_x, radius in zip(yellow_centers_y, yellow_centers_x, yellow_radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        valid_coords = np.logical_and(circy >= 0, circy < yellow_circles_image.shape[0]) & np.logical_and(circx >= 0, circx < yellow_circles_image.shape[1])
        yellow_circles_image[circy[valid_coords], circx[valid_coords]] = (0, 0, 255)  # yellow color for circles
        max_x = yellow_circles_image.shape[1]
        max_y = yellow_circles_image.shape[0]
        x1, y1, x2, y2 = max(0,center_x - 1.9*radius), max(0, center_y - 4.1*radius), min(max_x, center_x + 1.0*radius), min(max_y, center_y + (4.1)*radius)
        x1, x2, y1, y2 = round(x1), round(x2), round(y1), round(y2) 
        cropped_images.append(crop_image(image_array, x1, y1, x2, y2))
        bounding_boxes.append([x1, y1, x2, y2])
        labels.append("yellow") 
    return cropped_images, bounding_boxes, labels

In [61]:
def calculate_iou(boxA, boxB):
    # Extract the coordinates of the bounding boxes
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Calculate the intersection area
    intersection_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # Calculate the areas of both bounding boxes
    boxA_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # Calculate the union area
    union_area = float(boxA_area + boxB_area - intersection_area)

    # Calculate the IOU
    iou = intersection_area / union_area

    return iou

In [ ]:
unique_filenames = train_annotation_df['filename'].unique()
id_1 = 0
id_2 = 0

# Iterate over each unique filename
for filename in unique_filenames:
    # Filter rows with the current filename
    random_number = np.random.rand()
    if (random_number > 0.05):
        continue

    filtered_rows = train_annotation_df[train_annotation_df['filename'] == filename]
    img_path = filtered_rows.iloc[0]['image_path']
    img_name = filtered_rows.iloc[0]['filename']


    cropped_images, bounding_boxes, labels = generate_bouding_boxes(img_path + img_name)
    for bb_id, bb in enumerate(bounding_boxes):
        corrected = False
        for index, row in filtered_rows.iterrows():
            x1 = row['x1']
            y1 = row['y1']
            x2 = row['x2']
            y2 = row['y2']
            if (calculate_iou(bb, [x1, y1, x2, y2]) > 0.3):
                corrected = True

        label = "traffic_light" if corrected else "not_traffic_light"
        img_to_save = (cropped_images[bb_id])
        if not corrected:
            output_path = "../input/"+label+"/"+str(id_2)+".jpg"
            cv2.imwrite(output_path, img_to_save)
            id_2+=1

In [ ]:
unique_filenames = train_annotation_df['filename'].unique()
random.shuffle(unique_filenames)

count = {'green': 0, 'red': 0, 'yellow': 0}
for filename in unique_filenames:
    filtered_rows = train_annotation_df[train_annotation_df['filename'] == filename]
    img_path = filtered_rows.iloc[0]['image_path']
    img_name = filtered_rows.iloc[0]['filename']

    n_items = 0

    for index, row in filtered_rows.iterrows():
        x1 = row['x1']
        y1 = row['y1']
        x2 = row['x2']
        y2 = row['y2']
        target = row['target']
        target = color_map[target]

        random_number = np.random.rand()
        count[target]+= 1
        if (count[target] > 3000):
            continue

        img = cv2.imread(img_path + img_name)
        output_img = crop_image(img, x1, y1, x2, y2)
        output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)

        output_path = "../input/" + target + "/"+str(count[target])+".jpg"
        cv2.imwrite(output_path, output_img)
        break